# 誤差関数の変更による学習効果

---

## 目的

ネットワークの学習に使用する誤差関数（Loss関数）を変更した際に獲得される特徴表現（空間）を可視化し，その効果を確認する．

## 準備

### Google Colaboratoryの設定確認・変更
本チュートリアルではPyTorchを利用してニューラルネットワークの実装を確認，学習および評価を行います．
**GPUを用いて処理を行うために，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．**

## モジュールのインポート
はじめに必要なモジュールをインポートする．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`Use CUDA: True`と表示されれば，GPUを使用した計算をPyTorchで行うことが可能です．
Falseとなっている場合は，上記の「Google Colaboratoryの設定確認・変更」に記載している手順にしたがって，設定を変更した後に，モジュールのインポートから始めてください．

In [ ]:
# モジュールのインポート
from time import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## ネットワークモデルの定義

畳み込みニューラルネットワークを定義します．

ここでは，畳み込み層２層，全結合層３層から構成されるネットワークとします．

1層目の畳み込み層は入力チャンネル数が1，出力する特徴マップ数が16，畳み込むフィルタサイズが3x3です．
2層目の畳み込み層は入力チャネル数が16．出力する特徴マップ数が32，畳み込むフィルタサイズは同じく3x3です．
１つ目の全結合層は入力ユニット数は`7*7*32`とし，出力は1024としています．
次の全結合層入力，出力共に1024，出力層は入力が1024，出力が10です．
また，活性化関数として`self.act`にシグモイド関数を定義します．
さらに，プーリング処理を行うための`self.pool`を定義します．
ここでは，maxpoolingを使用します．
これらの各層の構成を`__init__`関数で定義します．

次に，`forward`関数では，定義した層を接続して処理するように記述します．
`forward`関数の引数`x`は入力データです．
それを`__init__`関数で定義した`conv1`に入力し，その出力を活性化関数である`self.act`に与えます．
そして，その出力を`self.pool`に与えて，プーリング処理結果を`h`として出力します．
2層目の畳み込み層でも同様の手順で処理を行います．

畳み込みを適用した後の特徴マップを全結合層へと入力して，識別結果を出力します．
まず．畳み込みによって得られた特徴マップの形状（チャンネルx縦x横）を1次元の配列へと変換します．
ここで，`view()`を用いることで，`h`の配列を操作します．引数として，変換したい配列のサイズを入力します．
まず一つ目の引数の`h.size()[0]`で，`h`の1次元目のサイズを取得し，変換後の配列の1次元目のサイズとして指定します．
二つ目の引数の`-1`で任意のサイズを指定します．
これにより，`h`を（バッチ数x任意の長さのデータ）の形状へ変換します．
変換した`h`を全結合層および活性化関数へと順次入力することで，最終的にクラススコアを返します．

In [ ]:
class CNN(nn.Module):
  def __init__(self, in_channels=1, hidden_dim=2, num_classes=10):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=5, stride=1, padding=2, bias=True)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2, bias=True)
    self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2, bias=True)
    self.conv4 = nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2, bias=True)
    self.conv5 = nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2, bias=True)
    self.conv6 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2, bias=True)

    self.act = nn.PReLU()

    self.gap = nn.AdaptiveAvgPool2d(1)
    self.fc1 = nn.Linear(128, hidden_dim)
    self.fc2 = nn.Linear(hidden_dim, num_classes)
  
  def forward(self, x):

    h = self.act(self.conv1(x))
    h = self.act(self.conv2(h))
    h = F.max_pool2d(h, 2, 2)
    h = self.act(self.conv3(h))
    h = self.act(self.conv4(h))
    h = F.max_pool2d(h, 2, 2)
    h = self.act(self.conv5(h))
    h = self.act(self.conv6(h))
    h = F.max_pool2d(h, 2, 2)

    h = self.gap(h).flatten(start_dim=1)
    h_out = self.fc1(h)
    out = self.fc2(h_out)
    return out, h_out

## Loss関数の作成

次に，Cross Entropy Loss以外のLoss関数を定義します．

In [ ]:
class CenterLoss(nn.Module):
  def __init__(self, num_classes=10, num_features=2, use_gpu=True):
    super().__init__()
    self.num_classes = num_classes
    self.feat_dim = num_features
    self.use_gpu = use_gpu

    if self.use_gpu:
      self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
    else:
      self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

  def forward(self, x, labels):
    batch_size = x.size(0)
    distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
              torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
    distmat.addmm_(x, self.centers.t(), beta=1, alpha=-2)

    classes = torch.arange(self.num_classes).long()
    if self.use_gpu: classes = classes.cuda()
    labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
    mask = labels.eq(classes.expand(batch_size, self.num_classes))

    dist = distmat * mask.float()
    loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size
    return loss


class PCLoss(nn.Module):
  def __init__(self, num_features=2, num_classes=10, use_gpu=True):
    super().__init__()
    self.num_classes = num_classes
    self.feat_dim = num_features
    self.use_gpu = use_gpu

    if self.use_gpu:
      self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
    else:
      self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))
  
  def forward(self, x, labels):
    batch_size = x.size(0)
    distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
              torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
    distmat.addmm_(x, self.centers.t(), beta=1, alpha=-2)

    classes = torch.arange(self.num_classes).long()
    if self.use_gpu: classes = classes.cuda()
    labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
    mask = labels.eq(classes.expand(batch_size, self.num_classes))

    inverse_mask = 1 - mask.float()
    dist = (distmat * inverse_mask).clamp(min=1e-12, max=1e+12)

    pos_c = mask.float().unsqueeze(2).repeat(1,1,self.feat_dim) * self.centers.unsqueeze(0).repeat(batch_size,1,1)
    neg_c = inverse_mask.unsqueeze(2).repeat(1,1,self.feat_dim) * self.centers.unsqueeze(0).repeat(batch_size,1,1)
    dist_c = torch.pow(pos_c - neg_c, 2).clamp(min=1e-12, max=1e+12)
    loss = (torch.sum(dist) + torch.sum(dist_c)) / (batch_size * (self.num_classes - 1))
    return loss

## 学習・評価の関数の定義

本ノートブックでは，用いる誤差関数を変更し，それぞれで獲得された特徴の分布を可視化します．
そのため，複数回学習と評価を行う必要があります．
そこで，ここではネットワークやデータセット，パラメータなどの必要な情報を引数として渡すことでネットワークの学習や評価を実行する関数を定義します．


### 学習用関数の定義

学習用の関数`training()`では，次のような引数を定義します．

* `epoch`: 現在の学習回数（表示用）
* `model`: 学習するネットワークモデル
* `dataloader`: 学習に使用するデータのDataLoader
* `xent`: CrossEntropyLossを計算するクラスインスタンス
* `model_optimizer`: ネットワークモデルを学習するための最適化関数（optimizer）
* `center`: Center Lossを計算するためのクラスインスタンス（Noneの場合はCenter Lossは計算されずに学習を行う）
* `pc`: PC Lossを計算するためのクラスインスタンス（Noneの場合はPC Lossは計算されずに学習を行う）
* `center_optimzer`: Center Lossの中心座標を更新するためのoptimizer（Noneの場合はCenter Lossは計算されずに学習を行う）
* `pc_optimizer`: PC Lossの中心座標を更新するためのoptimizer（Noneの場合はPC Lossは計算されずに学習を行う）
* `lambda_center`: Center Lossを計算する際のスケールパラメータ（重み）
* `lambda_pc`: PC Lossを計算する際のスケールパラメータ（重み）

このうち，Center LossとPC Lossを計算するための誤差関数とそのoptimizerについては，デフォルトの値を`None`としておき，
入力された場合のみ，学習に使用するように定義を行います．


### 評価用関数の定義

評価用の関数`evaluation()`では，引数として`model`, `dataloader`, `xent`を用意します．
各引数は，上の`training()`の場合と同様です．
こちらでは，学習済みのネットワークモデルと評価に使用するデータのDataLoader, クロスエントロピー誤差を計算するクラスインスタンスを入力して，学習済みのネットワークの精度を確認します．

In [ ]:
def training(epoch, model, dataloader, xent, model_optimizer, center=None, pc=None, center_optimizer=None, pc_optimizer=None, lambda_center=1, lambda_pc=0.0001):
  model.train()
  total = 0
  total_acc = 0
  for index, (x, y) in enumerate(dataloader):
    x, y = x.cuda(), y.cuda()

    logits, features = model(x)

    # Cross Entropy Lossの計算
    xent_loss = xent(logits, y)
    # Center Lossの計算（引数にCenter LossとOptimizerが設定されていれば）
    if center is not None and center_optimizer is not None:
        center_loss = center(features, y)
    else:
        center_loss = 0
    # PC Lossの計算（引数にPC LossとOptimizerが設定されていれば）
    if pc is not None and pc_optimizer is not None:
        pc_loss = pc(features, y)
    else:
        pc_loss = 0

    loss = xent_loss + lambda_center * center_loss - lambda_pc * pc_loss

    model_optimizer.zero_grad()
    loss.backward()
    model_optimizer.step()

    if center is not None and center_optimizer is not None:
        center_optimizer.zero_grad()
        center_optimizer.step()

    if pc is not None and pc_optimizer is not None:
        pc_optimizer.zero_grad()
        for param in pc.parameters():
            param.grad.data *= (1/lambda_pc)
        pc_optimizer.step()

    if index % 100 == 0:
      num_correct = torch.argmax(torch.softmax(logits, dim=1), dim=1).eq(y).sum().item()
      total += x.size(0)
      total_acc += num_correct
      print('{} epoch [{}/{}] | '
           'Loss: {:.3f} | xent: {:.3f} | center: {:.3f} | pc: {:.3f} | '
           'Acc: {:.3f} (avg: {:.3f})'.format(epoch, index, len(dataloader), loss, xent_loss, center_loss, pc_loss, num_correct / x.size(0), total_acc / total))


def evaluation(model, dataloader, xent):
  model.eval()
  total = 0
  total_loss = 0
  total_correct = 0
  for index, (x, y) in enumerate(dataloader):
    x, y = x.cuda(), y.cuda()

    with torch.no_grad():
      logits, _ = model(x)
      loss = xent(logits, y)

    total += x.size(1)
    num_correct = torch.argmax(torch.softmax(logits, dim=1), dim=1).eq(y).sum().item()
    total_correct += num_correct
    total_loss += loss.item()
    # if index % 10 == 0:
    #   print('Test [{}/{}] | Loss: {:.4f} (avg: {:.4f}) | Acc: {:.4f} (avg: {:.4f})'.format(index, len(dataloader.dataset), loss.item(), total_loss/total, 100*(num_correct/x.size(0)), 100*(total_correct/total)))
  return total_loss/total, 100*(total_correct/total)

## 共通するパラメータの定義

実際の学習を行う前に，用いる誤差関数にかかわらず，同じ設定のパラメータを事前に設定します．

また，学習と評価に使用するデータ（DataLoader）を準備します．
今回はMNISTデータセットを使用して実験を行います．

In [ ]:
num_classes = 10
hidden_dim = 2
epochs = 100
batch_size = 100

train_dataset = datasets.MNIST('./root', download=True, train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transforms.ToTensor())
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, shuffle=True, drop_last=False, batch_size=1)

## 1. CrossEntropy誤差のみで学習した場合


In [ ]:
lr = 0.001
momentum = 0.9

model_xent = CNN(in_channels=1, hidden_dim=hidden_dim, num_classes=10).cuda()
xent = nn.CrossEntropyLoss().cuda()

model_optimizer = optim.SGD(model_xent.parameters(), lr=lr, momentum=momentum)

# 学習
for epoch in range(epochs):
  training(epoch, model_xent, train_dataloader, xent, model_optimizer)

# 評価
test_loss, test_acc = evaluation(model_xent, test_dataloader, xent)
print('Evaluation result: Loss {:.4f}, Acc {:.4f}'.format(test_loss, test_acc))

### 特徴の分布の可視化

In [ ]:
coords = np.zeros((len(test_dataloader.dataset), 2))
labels = np.zeros((len(test_dataloader.dataset)))
cnt = 0

model_xent.eval()
for (x, y) in test_dataloader:
  x = x.cuda()
  batch = x.size(0)
  _, features = model_xent(x)
  coords[cnt:cnt+batch] = features.squeeze().data.cpu().numpy()
  labels[cnt:cnt+batch] = y.data.numpy()
  cnt += batch

mpl_colorlist = plt.rcParams['axes.prop_cycle'].by_key()['color']
colorlist = [mpl_colorlist[int(idx)] for idx in labels]
xcoords, ycoords = coords[:, 0], coords[:, 1]
xmax, xmin = xcoords.max(), xcoords.min()
ymax, ymin = ycoords.max(), ycoords.min()
xcoords = (xcoords - xmin) / (xmax - xmin)
ycoords = (ycoords - ymin) / (ymax - ymin)

plt.scatter(xcoords, ycoords, color=colorlist, s=0.4)
plt.show()

## 2. Cross Entropy Loss + Center Lossを用いて学習した場合

In [ ]:
lr = 0.001
momentum = 0.9
center_lr = 0.5
lambda_center = 1

model_xent_center = CNN(in_channels=1, hidden_dim=hidden_dim, num_classes=10).cuda()
xent = nn.CrossEntropyLoss().cuda()
center = CenterLoss(num_classes=num_classes, num_features=hidden_dim, use_gpu=True)

model_optimizer = optim.SGD(model_xent_center.parameters(), lr=lr, momentum=momentum)
center_optimizer = optim.SGD(center.parameters(), lr=center_lr)

# 学習
for epoch in range(epochs):
  training(epoch, model_xent_center, train_dataloader, xent, model_optimizer,
           center=center, center_optimizer=center_optimizer, lambda_center=lambda_center)

# 評価
test_loss, test_acc = evaluation(model_xent_center, test_dataloader, xent)
print('Evaluation result: Loss {:.4f}, Acc {:.4f}'.format(test_loss, test_acc))

### 可視化


In [ ]:
coords = np.zeros((len(test_dataloader.dataset), 2))
labels = np.zeros((len(test_dataloader.dataset)))
cnt = 0

model_xent_center.eval()
for (x, y) in test_dataloader:
  x = x.cuda()
  batch = x.size(0)
  _, features = model_xent_center(x)
  coords[cnt:cnt+batch] = features.squeeze().data.cpu().numpy()
  labels[cnt:cnt+batch] = y.data.numpy()
  cnt += batch

mpl_colorlist = plt.rcParams['axes.prop_cycle'].by_key()['color']
colorlist = [mpl_colorlist[int(idx)] for idx in labels]
xcoords, ycoords = coords[:, 0], coords[:, 1]
xmax, xmin = xcoords.max(), xcoords.min()
ymax, ymin = ycoords.max(), ycoords.min()
xcoords = (xcoords - xmin) / (xmax - xmin)
ycoords = (ycoords - ymin) / (ymax - ymin)

plt.scatter(xcoords, ycoords, color=colorlist, s=0.4)
plt.show()

## Cross Entropy Loss + Center Loss + PC Lossを用いて学習した場合


In [ ]:
lr = 0.001
momentum = 0.9
# Center Lossのパラメータ
center_lr = 0.5
lambda_center = 1
# PC Lossのパラメータ
pc_lr = 0.01
lambda_pc = 0.0001

model_xent_center_pc = CNN(in_channels=1, hidden_dim=hidden_dim, num_classes=10).cuda()
xent = nn.CrossEntropyLoss().cuda()
center = CenterLoss(num_classes=num_classes, num_features=hidden_dim, use_gpu=True)
pc = PCLoss(num_classes=num_classes, num_features=hidden_dim, use_gpu=True)

model_optimizer = optim.SGD(model_xent_center_pc.parameters(), lr=lr, momentum=momentum)
center_optimizer = optim.SGD(center.parameters(), lr=center_lr)
pc_optimizer = optim.SGD(pc.parameters(), lr=pc_lr)

# 学習
for epoch in range(epochs):
  training(epoch, model_xent_center_pc, train_dataloader, xent, model_optimizer,
           center=center, pc=pc, center_optimizer=center_optimizer, pc_optimizer=pc_optimizer,
           lambda_center=lambda_center, lambda_pc=lambda_pc)

# 評価
test_loss, test_acc = evaluation(model_xent_center_pc, test_dataloader, xent)
print('Evaluation result: Loss {:.4f}, Acc {:.4f}'.format(test_loss, test_acc))

### 可視化

In [ ]:
coords = np.zeros((len(test_dataloader.dataset), 2))
labels = np.zeros((len(test_dataloader.dataset)))
cnt = 0

model_xent_center_pc.eval()
for (x, y) in test_dataloader:
  x = x.cuda()
  batch = x.size(0)
  _, features = model_xent_center_pc(x)
  coords[cnt:cnt+batch] = features.squeeze().data.cpu().numpy()
  labels[cnt:cnt+batch] = y.data.numpy()
  cnt += batch

mpl_colorlist = plt.rcParams['axes.prop_cycle'].by_key()['color']
colorlist = [mpl_colorlist[int(idx)] for idx in labels]
xcoords, ycoords = coords[:, 0], coords[:, 1]
xmax, xmin = xcoords.max(), xcoords.min()
ymax, ymin = ycoords.max(), ycoords.min()
xcoords = (xcoords - xmin) / (xmax - xmin)
ycoords = (ycoords - ymin) / (ymax - ymin)

plt.scatter(xcoords, ycoords, color=colorlist, s=0.4)
plt.show()

## 課題

1. Lossのパラメータを変更して，学習後の特徴の分布がどのように変化するかを確認しましょう．
